In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import progressbar as progressbar

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
print(f'TF version: {tf.__version__}')
# print(tf.config.list_physical_devices())

TF version: 2.0.0


# Read Data

In [8]:
from Data_prep import leave_last_out

In [9]:
path = 'C:/Users/robin.opdam/Google Drive/Thesis (Msc)/Thesis_shared_files/'
# path = '/Users/Robin/Google Drive/Thesis (Msc)/Thesis_shared_files/'
# path = '../'

In [10]:
data_path = 'Data/Amazon/'
file_name = 'am_20k_users'
# file_name = 'am_like_ml'

# data_path = 'Data/ML/'
# file_name = 'ml_1m'

In [11]:
df = pd.read_pickle(path + data_path + file_name)
total_items = df.item_id.nunique()
total_users = df.user_id.nunique()

In [12]:
# MF based Data
train_set_mf = pd.read_pickle(path + data_path + file_name + '_train_mf')
val_set_mf = pd.read_pickle(path + data_path + file_name + '_val_mf')
test_set_mf = pd.read_pickle(path + data_path + file_name + '_test_mf')

In [13]:
# Eval
steps = 5
rank_at = 20

# Creating Samples BPR

In [14]:
bpr_params = {
    "nolf": 8, #Size of latent feature vectors
    "n_iterations": 25, #around 20 is sufficient
    "sample_perc": 0.5,
    "seed": 1234,
    "alpha": 0.08, # Learning rate: Amazon_01 best: 0.08
    "rho": 1.05, # Bold driver increase alpha
    "sigma": 0.55, # Bold driver decrease alpha

    "reg_user": 0.1, # Am_40k best: 0.1 # ML_1 best: 0
    "reg_item": 0.1, # Am_40k best: 0.1 # ML_1 best: 0
    }

In [15]:
data_paths = ['Data/Amazon/']#['Data/ML/', 'Data/Amazon/', 'Data/Amazon/']
file_names = ['am_20k_users']#['ml_1m', 'am_like_ml', 'am_20k_users']
sample_percs = [0.5]

for file_name, data_path in zip(file_names, data_paths):
    # MF based Data
    train_set_mf = pd.read_pickle(path + data_path + file_name + '_train_mf')
    val_set_mf = pd.read_pickle(path + data_path + file_name + '_val_mf')
    test_set_mf = pd.read_pickle(path + data_path + file_name + '_test_mf')
    
    for sample_perc in sample_percs:
        bpr_params['sample_size'] = sample_perc * len(train_set_mf)
        bpr = BPR(total_users, total_items, bpr_params)

        samples_name = file_name + '_bpr_samples_' + str(sample_perc)
        samples = bpr.create_samples(train_set_mf)
        samples = np.array(samples)
        np.save(path + data_path + 'Samples/' + samples_name, samples)

Creating 25 samples of length 89654


100% |########################################################################|


# Creating sampes NCF

In [43]:
run_num = 'am_40k_nolf_8'

In [44]:
GMF_params = {
    'learning_rate': 0.001,
    'batch_size': 256,
    'nolf': 8,
    'regs': [0,0],
    'epochs': 20,
    'sample_size': len(train_set),#int(0.5*len(train_set.user_id.unique())),
    'num_neg': 4,
    'ckpt_dir': f'../NeuMF_storage/GMF_ckpts_{run_num}/ckpts',
    'optimizer':'Adam'
}

## Create Samples (MP)

## Random User sampling

In [45]:
import multiprocessing as mp        
def create_sample(user_items, train_users, train_items, params, num_processes):
    samples_sizes_split = np.array_split(np.array(range(params['sample_size'])),num_processes)
    args = []
    for samples_size in samples_sizes_split:
        args.append((user_items, train_users, train_items, len(samples_size), params['num_neg']))
    with mp.Pool(processes=num_processes) as pool:
        results = pool.starmap(create_sample_worker, args)

    user_inputs, item_inputs, labels = [], [], []
    for res_epochs in results: 
        user_inputs.extend(res_epochs['u'])
        item_inputs.extend(res_epochs['i'])
        labels.extend(res_epochs['l'])

    return user_inputs, item_inputs, labels


def create_sample_worker(user_items, train_users, train_items, sample_size, num_neg):
    user_inputs, item_inputs, labels = [], [], []
    for s in range(sample_size):
        # Add positive item
        u = np.random.choice(train_users)
        u_items = user_items[u]
        i = np.random.choice(u_items)

        user_inputs.append(u)
        item_inputs.append(i)
        labels.append(1)

        # Add negative item
        for i in range(num_neg):
            j = np.random.choice(train_items)
            while j in u_items:  # neg item j cannot be in the set of pos items of user u
                j = np.random.choice(train_items)

            user_inputs.append(u)
            item_inputs.append(j)
            labels.append(0)

    return {'u':user_inputs, 'i':item_inputs, 'l':labels}

## Single user sampling

In [46]:
import multiprocessing as mp        
def create_user_sample(user_items, train_users, train_items, params, num_processes):
    users_splits = np.array_split(np.array(train_users),num_processes)
    args = []
    for user_split in users_splits:
        args.append((user_items, user_split, train_items, params['num_neg']))
    with mp.Pool(processes=num_processes) as pool:
        results = pool.starmap(create_user_sample_worker, args)

    user_inputs, item_inputs, labels = [], [], []
    for res_epochs in results: 
        user_inputs.extend(res_epochs['u'])
        item_inputs.extend(res_epochs['i'])
        labels.extend(res_epochs['l'])

    return user_inputs, item_inputs, labels


def create_user_sample_worker(user_items, train_users, train_items, num_neg):
    user_inputs, item_inputs, labels = [], [], []
    for user in train_users:
        # All positive items for this user
        u_items = user_items[user]
        
        # Per positive item, sample num_neg negative items
        for u_item in u_items:
            pos_item = np.random.choice(u_items)

            user_inputs.append(user)
            item_inputs.append(pos_item)
            labels.append(1)

            # Add negative item
            for i in range(num_neg):
                neg_item = np.random.choice(train_items)
                while neg_item in u_items:  # neg item j cannot be in the set of pos items of user u
                    neg_item = np.random.choice(train_items)

                user_inputs.append(user)
                item_inputs.append(neg_item)
                labels.append(0)

    return {'u':user_inputs, 'i':item_inputs, 'l':labels}

In [47]:
user_items = train_set.groupby('user_id')['item_id'].apply(list)
train_users = train_set.user_id.unique()
train_items = train_set.item_id.unique()

In [48]:
store_path = path + data_path + 'Samples/' + file_name + '_samples_' + str(GMF_params['num_neg']) + '_neg'
sample_name = file_name + '_sample'

In [49]:
%%time
import csv
num_processes = mp.cpu_count()
val_metrics = []
for epoch in range(30):
    print(f'Epoch: {epoch}')
    user_inputs, item_inputs, labels = create_user_sample(user_items, train_users, train_items, GMF_params, num_processes)
    samples = [user_inputs, item_inputs, labels]
    file = open(f'{store_path}/{sample_name}_{epoch}.csv', 'w+', newline='')
    with file:
        write = csv.writer(file)
        write.writerows(samples)

Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
CPU times: user 52.6 s, sys: 6.03 s, total: 58.6 s
Wall time: 4min 9s


In [27]:
file_name

'ml_1m'

In [24]:
len(samples[0])

4993545

In [18]:
users_split = np.array_split(np.array(train_users),num_processes)

(3334,)

In [43]:
import csv

file = open(f'../NeuMF_storage/samples/ml_1m_sample_test.csv', 'w+', newline='')

with file:
    write = csv.writer(file)
    write.writerows(samples)

## Read Sample

In [16]:
%%time
with open('samples.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    samples = list(csv_reader)

CPU times: user 965 ms, sys: 290 ms, total: 1.26 s
Wall time: 1.31 s
